## Import das bibliotecas utilizadas


In [12]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima



### Carregamento dos dados para a parte de Modelagem

In [13]:
df_transposed_copy_inputting_v19 = pd.read_csv(r'../../Projeto/Data/Output/df_transposed_copy_inputting_v19.csv')


## Modelagem

Modelos de Forecasting são divididos em partes pré e pós período desejado para previsão. Os períodos utilizados devem ser equiparáveis ao tipo de questão analisada.

In [14]:
df_model = df_transposed_copy_inputting_v19.copy()

In [15]:
df_model['year'] = pd.to_datetime(df_model['year'], format='ISO8601')


In [16]:
df_model.set_index('year', inplace = True)
df_model.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49 entries, 1980-01-01 to 2028-01-01
Columns: 228 entries, Afghanistan to World
dtypes: float64(228)
memory usage: 87.7 KB


#### Training and Test set

In [17]:
test_years = 5
train_data = df_model.iloc[:-test_years, :]

In [18]:
test_data = df_model.iloc[-test_years:, :]
test_data.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2024-01-01 to 2028-01-01
Columns: 228 entries, Afghanistan to World
dtypes: float64(228)
memory usage: 8.9 KB


### Modelo SARIMAX sem variável exógena

In [20]:
df_pred_auto_sarimax_no_exog = pd.DataFrame()
for country in df_model.columns:
        model_sarimax_no_exog = auto_arima(train_data[country],
                m=4,
                test='adf',
                seasonal=True,
                stepwise=True,
                #d=0,
                #start_p=0,
                #max_p=1,
                #start_q=0,
                #max_q=1,
        )
        predictions_sarimax_no_exog = model_sarimax_no_exog.predict(n_periods = len(test_data[country])).rename("SARIMAX Non Exog")
        df_pred_auto_sarimax_no_exog = pd.concat([df_pred_auto_sarimax_no_exog, pd.DataFrame(predictions_sarimax_no_exog.values, columns=[country])], axis=1)
df_pred_auto_sarimax_no_exog.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,...,Emerging and Developing Europe,Emerging market and developing economies,Euro area,European Union,Latin America and the Caribbean,Major advanced economies (G7),Middle East and Central Asia,Other advanced economies,Sub-Saharan Africa,World
0,4.716307,3.738784,5.730262,17.358588,2.431248,3.889696,-0.093307,5.5,2.369578,3.560694,...,1.990091,3.898848,1.762987,1.474017,-0.957572,2.086364,2.9,2.840787,3.296295,2.880376
1,6.623662,2.832769,1.779494,14.977775,2.431248,3.386085,3.584008,5.5,2.369578,2.707185,...,1.725775,3.898848,1.762987,1.978909,3.344497,2.086364,2.9,2.885672,3.296295,3.450667
2,5.131860,2.832769,1.866865,13.016893,2.431248,3.386085,3.064900,5.5,2.369578,3.230111,...,1.680279,3.898848,1.762987,1.878592,2.883838,2.086364,2.9,2.927956,3.296295,3.181182
3,7.627398,2.832769,2.045162,25.204504,2.431248,3.386085,1.799113,5.5,2.369578,2.909726,...,1.336253,3.898848,1.762987,1.794893,1.899359,2.086364,2.9,2.967791,3.296295,2.657519
4,8.032991,2.832769,2.293944,22.505046,2.431248,3.386085,1.749198,5.5,2.369578,3.106019,...,1.922639,3.898848,1.762987,1.807118,0.998756,2.086364,2.9,3.005319,3.296295,2.202266


### Save dataframe

In [22]:
## Export forecast

df_pred_auto_sarimax_no_exog.to_csv('../../Projeto/Data/Output/predict.csv', index = True)